# Suspicious Svchost Process
Detects a suspicious svchost process start

## Rule Content
```
- title: Suspicious Svchost Process
  id: 01d2e2a1-5f09-44f7-9fc1-24faa7479b6d
  status: experimental
  description: Detects a suspicious svchost process start
  tags:
  - attack.defense_evasion
  - attack.t1036
  author: Florian Roth
  date: 2017/08/15
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image: '*\svchost.exe'
    filter:
      ParentImage:
      - '*\services.exe'
      - '*\MsMpEng.exe'
      - '*\Mrt.exe'
      - '*\rpcnet.exe'
      - '*\svchost.exe'
    filter_null:
      ParentImage: null
    condition: selection and not filter and not filter_null
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\svchost.exe AND (NOT (process_parent_path.keyword:(*\\services.exe OR *\\MsMpEng.exe OR *\\Mrt.exe OR *\\rpcnet.exe OR *\\svchost.exe)))) AND (NOT (NOT _exists_:process_parent_path)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()